Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## 1. Загрузка данных

In [1]:
# Базовая загрузка бибиотек
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
data = pd.read_csv('/datasets/insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
# В рамках задачи считаем, что дополнительная обработка данных нам не нужна
# Разобьем наши данные на признаки и целевой показатель
features = data.drop(columns='Страховые выплаты')
target = data['Страховые выплаты']

features.shape

(5000, 4)

In [5]:
target.shape

(5000,)

## 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$a = Xw$$

Задача обучения:

$$w = \arg\min_w MSE(Xw, y)$$

Формула обучения:

$$w = (X^T X)^{-1} X^T y$$

## 3. Ответим на поставленный вопрос (с алгоритмом преобразования)

Что произойдет, если признаки умножить на обратимую матрицу. Изменится ли качество линейной регрессии?

Очевидных факторов, указывающих на правильный ответ (кроме самой постановки вопроса), я не увидел. Поэтому попробую все же проверить гипотезу.

Для ответа необходимо доказать (или опровергнуть), что предсказания $a$ на исходных признаках будут равны $a_2$ вычесленному с учетом умножения признаков на заведомо обратимую матрицу (судя по исходным данным, это будет $P$).

1. Вычислим $w_2$. Подставим в выражение:

$$w_2 = ((XP)^T XP)^{-1} (XP)^T y$$

2. Раскроем скобки для транспонирования по свойству $(AB)^T = B^T A^T$

$$w_2 = (P^T X^T XP)^{-1} P^T X^T y$$

3. Вынесем P за скобки по свойству $(AB)^{-1} = B^{-1} A^{-1}$

$$w_2 = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y$$

4. Т.к. умножение матрицы на обратимую равно E, можем сократить часть $(P^T)^{-1} P^T$ до $E$ и сразу умножить на $X^T$

$$w_2 = P^{-1} (X^T X)^{-1} X^T y$$

5. Вспомним, что $w = (X^T X)^{-1} X^T y$ и произведем замену

$$w_2 = P^{-1} w$$

Теперь проверим признаки

1. Запишем формулу

$$a_2 = XPw_2$$

2. Подставим получившиеся значения

$$a_2 = XP P^{-1} w$$

3. Опять же, вспомним, что умножение на обратную матрицу даст $E$

$$a_2 = Xw$$

**4. Учитывая, что $a = Xw$, получаем:**

$$a_2 = a$$

### Вывод

Теперь мы точно можем сказать, что при умножении признаков на обратимую матрицу, качество линейной регрессии не изменится.

## 3. Алгоритм преобразования

Похоже этот шаг я объеденил с предыдущим. :)

    1. создаём случайную матрицу
    2. гарантируем ее обратимость
    3. создаём новую матрицу признаков
    4. ищем метрики качества как для исходных признаков, так и сгенерированных
    5. делаем вывод

## 4. Проверка алгоритма

In [6]:
# Запрограммируем этот алгоритм, применив матричные операции

def generate_invertible_matrix(size):
    try:
        matrix = np.random.normal(size=(size, size))
        # проверим матрицу на обратимость, если нет, пробуем сгенерировать еще раз
        # таким образом гарантируем, что матрица стопроцентно будет обратимой
        np.linalg.inv(matrix)
    except np.linalg.LinAlgError:
        matrix = generate_invertible_matrix(size)
    
    return matrix

def make_new_features(features):
    size = features.shape[1]
    P = generate_invertible_matrix(size)
    return features.dot(P)
    
new_features = make_new_features(features)

# Проверим, что все ок
features.shape, new_features.shape

((5000, 4), (5000, 4))

In [7]:
# Обучим 2 линейные регрессии на наших признаках

model = LinearRegression()

model.fit(features, target)
predict = model.predict(features)

model.fit(new_features, target)
new_predict = model.predict(new_features)

# Сравним R2
print('R2 features = ', r2_score(target, predict))
print('R2 new_features = ', r2_score(target, new_predict))

R2 features =  0.42494550286668
R2 new_features =  0.4249455028666771


# Общий вывод

    1. Мы изучили данные
    2. Доказали, что при умножении признаков на обратимую матрицу качество линейной регрессии не изменится и обосновали это заключение
    3. Помимо этого, мы также запрограммировали этот алгоритм и проверили качество линейной регрессии после преобразования, убедившись, что оно не изменилось.
    
    Таким образом, у нас получилось создать метод преобразования данных, по которому будет сложно восстановить персональную информацию, но который не будет влиять на качество линейной регрессии. Что и требовалось получить.)


